# Notebook to plot the helicity angles of $a_{0}$

In [2]:
import ROOT
import numpy as np
import my_library.common_analysis_tools as ct
import my_library.constants as ct
import my_library.kinematic_cuts as kcuts

In [8]:
def get_gamma_cm(ey, s):
    return (ey + 0.938272)/np.sqrt(s)


def get_beta_cm(ey):
    return ey/(ey + 0.938272)


def get_beta(p, e):
    return p/e


def get_gamma(e, m):
    return e/m


def get_cm_rotation(ey, s):
    gamma = get_gamma_cm(ey, s)
    beta = get_beta_cm(ey)
    return np.array([[gamma, 0, 0, -beta*gamma,],
                     [0, 1, 0, 0],
                     [0, 0, 1, 0],
                     [-beta*gamma, 0, 0, gamma]])


def get_cm_vector(e, px, py, pz, ey, s, verbose=False):
    gamma = get_gamma_cm(ey, s)
    beta = get_beta_cm(ey)
    vec_cm_rotaton = np.array([[gamma, 0, 0, -beta*gamma,],
                     [0, 1, 0, 0],
                     [0, 0, 1, 0],
                     [-beta*gamma, 0, 0, gamma]])
    vec_lab = np.array([e, px, py, pz]).reshape(-1, 1)

    if verbose:
        print(f"|{vec_lab[0][0]}| |{vec_cm_rotaton[0][0]} {vec_cm_rotaton[0][1]} {vec_cm_rotaton[0][2]} {vec_cm_rotaton[0][3]}|\n" 
            f"|{vec_lab[1][0]}| |{vec_cm_rotaton[1][0]} {vec_cm_rotaton[1][1]} {vec_cm_rotaton[1][2]} {vec_cm_rotaton[1][3]}|\n"
            f"|{vec_lab[2][0]}| |{vec_cm_rotaton[2][0]} {vec_cm_rotaton[2][1]} {vec_cm_rotaton[2][2]} {vec_cm_rotaton[2][3]}|\n"
            f"|{vec_lab[3][0]}| |{vec_cm_rotaton[3][0]} {vec_cm_rotaton[3][1]} {vec_cm_rotaton[3][2]} {vec_cm_rotaton[3][3]}|"
            )
        
    return (vec_cm_rotaton @ vec_lab)


def get_rest_frame_vec(p4_decay_cm, p4_rest_cm, m_rest):
    """e in 0th element, px in 1st, py in 2nd, pz in 3rd"""
    beta = get_beta(np.sqrt(p4_rest_cm[1]*p4_rest_cm[1] + p4_rest_cm[2]*p4_rest_cm[2] + p4_rest_cm[3]*p4_rest_cm[3]), p4_rest_cm[0], p4_rest_cm[0])
    gamma = get_gamma(p4_rest_cm[0], m_rest)
    beta_x = get_beta(p4_rest_cm[1], p4_rest_cm[0])
    beta_y = get_beta(p4_rest_cm[2], p4_rest_cm[0])
    beta_z = get_beta(p4_rest_cm[3], p4_rest_cm[0])

    p4_decay_cm.reshape(-1, 1)

    rest_rotation = np.array([[gamma,         -gamma*beta_x,                             -gamma*beta_y,                             -gamma*beta_z                            ],
                              [-gamma*beta_x, 1 + (gamma - 1)*beta_x*beta_x/(beta*beta), (gamma - 1)*beta_x*beta_y/(beta*beta),     (gamma - 1)*beta_x*beta_z/(beta*beta)    ],
                              [-gamma*beta_y, (gamma - 1)*beta_y*beta_x/(beta*beta),     1 + (gamma - 1)*beta_y*beta_y/(beta*beta), (gamma - 1)*beta_y*beta_z/(beta*beta)    ],
                              [-gamma*beta_z, (gamma - 1)*beta_z*beta_x/(beta*beta),     (gamma - 1)*beta_z*beta_y/(beta*beta),     1 + (gamma - 1)*beta_z*beta_z/(beta*beta)]])

    return rest_rotation @ p4_decay_cm


